In [11]:
pip install requests beautifulsoup4 pandas


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
from bs4 import BeautifulSoup

def fetch_amazon_bestsellers(year, page_num=1):
    base_url = f'https://www.amazon.com/gp/bestsellers/{year}/books'
    url = f'{base_url}?page={page_num}'
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    # Send request to the page
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve page {url}. Status code: {response.status_code}")
        return None
    
    # Parse the page with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    books_info = []

    # Loop through each book on the page
    for book in soup.find_all('div', {'data-asin': True}):
        book_info = {}
        
        # Extracting the book title
        title_tag = book.find('div', {'class': '_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y'})
        book_info['title'] = title_tag.get_text(strip=True) if title_tag else 'N/A'

        # Extracting the author name
        author_tag = book.find('div', {'class': 'a-row a-size-small'}).find('a')
        book_info['author'] = author_tag.get_text(strip=True) if author_tag else 'N/A'

        # Extracting the rating
        rating_tag = book.find('a', {'class': 'a-link-normal', 'title': True})
        book_info['rating'] = rating_tag['title'] if rating_tag else 'N/A'

        # Extracting the price
        price_tag = book.find('span', {'class': 'p13n-sc-price'})
        book_info['price'] = price_tag.get_text(strip=True) if price_tag else 'N/A'

        # Extracting ASIN
        book_info['asin'] = book.get('data-asin', 'N/A')
        
        books_info.append(book_info)

    return books_info

# Example usage for fetching bestsellers from 2023, page 3
bestsellers_2023 = fetch_amazon_bestsellers(2023, page_num=3)

# Print the result for inspection
for book in bestsellers_2023:
    print(book)


{'title': 'Atomic Habits: An Easy & Proven Way to Build Good Habits & Break Bad Ones', 'author': 'James Clear', 'rating': '4.8 out of 5 stars, 139,854 ratings', 'price': 'N/A', 'asin': '0735211299'}
{'title': 'Iron Flame (The Empyrean, 2)', 'author': 'Rebecca Yarros', 'rating': '4.7 out of 5 stars, 331,364 ratings', 'price': '$9.99', 'asin': '1649374178'}
{'title': 'Spare', 'author': 'Prince Harry  The Duke of Sussex', 'rating': '4.5 out of 5 stars, 113,223 ratings', 'price': 'N/A', 'asin': '0593593804'}
{'title': 'Fourth Wing (The Empyrean, 1)', 'author': 'Rebecca Yarros', 'rating': '4.8 out of 5 stars, 399,050 ratings', 'price': '$20.58', 'asin': '1649374046'}
{'title': 'The Woman in Me', 'author': 'Britney Spears', 'rating': '4.4 out of 5 stars, 49,585 ratings', 'price': 'N/A', 'asin': '1668009048'}
{'title': 'Lessons in Chemistry: A Novel', 'author': 'Bonnie Garmus', 'rating': '4.6 out of 5 stars, 313,494 ratings', 'price': 'N/A', 'asin': '038554734X'}
{'title': 'The 48 Laws of Pow

In [24]:
pip install selenium



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#IT WORKED

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd

# Configure headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    # Try breadcrumbs method first
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    # Extract the customer average rating
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    if rating_tag:
        return rating_tag.get_text(strip=True)  # Extract the rating (e.g., 4.8)
    return 'N/A'

def extract_price(soup):
    price_span = soup.select_one("span.a-price span.a-offscreen")
    if price_span:
        return price_span.get_text(strip=True)
    return 'N/A'

def scrape_amazon_bestsellers(year):
    print(f"Scraping data for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 3):  # Scrape 2 pages as an example
        url = f"{base_url}?page={page}"
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y").get_text(strip=True)
                author = card.select_one("div.a-row.a-size-small").get_text(strip=True)
                book_url_tag = card.select_one("a.a-link-normal")
                book_url = "https://www.amazon.com" + book_url_tag['href']

                # Visit individual book page to get genre, rating, and price
                driver.get(book_url)
                time.sleep(1)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')
                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)  # Extract rating using updated function
                price = extract_price(book_soup)

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': year,
                    'url': book_url
                })

            except Exception as e:
                print(f"Error scraping a book: {e}")
                continue

    return book_data

# Collect for a single year (e.g., 2023)
books_2023 = scrape_amazon_bestsellers(2023)

# Save to CSV
df = pd.DataFrame(books_2023)
df.to_csv("amazon_bestsellers_2023.csv", index=False)
driver.quit()
print("Saved data to amazon_bestsellers_2023.csv")


There was an error managing chromedriver (error decoding response body); using driver found in the cache


Scraping data for 2023...
Saved data to amazon_bestsellers_2023.csv


In [86]:
# 2015 FINAL COUNTDOWN

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

# Configure headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    # Try breadcrumbs method first
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    # Extract the customer average rating
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    if rating_tag:
        return rating_tag.get_text(strip=True)  # Extract the rating (e.g., 4.8)
    return 'N/A'

def extract_price(soup):
    # Extract price from the page
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        return price_span.get_text(strip=True) + "." + soup.select_one("span.a-price-fraction").get_text(strip=True)
    return 'N/A'  # Return 'N/A' if no price is found

def extract_reviews(soup):
    # Extract number of customer reviews
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    if reviews_tag:
        return reviews_tag.get_text(strip=True)
    return 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        # Use Open Library API to fetch book data based on the title
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()

        # Check if data is available
        if data['docs']:
            book_info = data['docs'][0]  # Take the first result (most relevant)
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception as e:
        print(f"Error fetching data from Open Library API for title: {title}")
        return 'Unknown', 'Unknown'

def extract_year_from_subtitle(soup):
    # Look for the product subtitle which contains the publishing date
    subtitle_tag = soup.select_one("span#productSubtitle")
    if subtitle_tag:
        subtitle_text = subtitle_tag.get_text(strip=True)
        # Extract the year from the subtitle text
        match = re.search(r"(\d{4})", subtitle_text)
        if match:
            return match.group(1)
    return 'Unknown'  # If no year is found, return 'Unknown'

def extract_year_and_price_from_amazon(soup):
    # Look for the div containing both the price and the potential year in the page structure
    price_div = soup.select_one("div#corePriceDisplay_desktop_feature_div")
    if price_div:
        # Extract the price
        price_tag = price_div.select_one("span.a-price span.a-offscreen")
        price = price_tag.get_text(strip=True) if price_tag else 'N/A'

        # Extract year from the text if available
        year_tag = price_div.select_one("span.a-size-base.a-color-secondary")
        if year_tag:
            year = year_tag.get_text(strip=True)
            # Check if the year and price are mixed (price + year)
            match = re.search(r"(\d{4})", year)
            if match:
                year = match.group(1)
            return year, price
    return 'Unknown', 'N/A'

def scrape_amazon_bestsellers(year):
    print(f"Scraping data for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 3):  # Scrape 2 pages as an example
        url = f"{base_url}?page={page}"
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y").get_text(strip=True)
                author = card.select_one("div.a-row.a-size-small").get_text(strip=True)
                book_url_tag = card.select_one("a.a-link-normal")
                book_url = "https://www.amazon.com" + book_url_tag['href']

                # Visit individual book page to get genre, rating, and price
                driver.get(book_url)
                time.sleep(1)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')
                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)  # Extract rating using updated function
                reviews = extract_reviews(book_soup)  # Extract the number of reviews

                # Fetch publisher and published year using Open Library API
                publisher, published_year = fetch_book_info_from_openlibrary(title)

                # If publisher and year are still unknown, try to extract from Amazon
                if published_year == 'Unknown':
                    # Extract year from the product subtitle
                    published_year = extract_year_from_subtitle(book_soup)
                    price = extract_price(book_soup)  # Fetch the price separately if not found together
                else:
                    price = extract_price(book_soup)  # Fetch the price separately if not found together

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping a book: {e}")
                continue

    return book_data

# Collect for a single year (e.g., 2015)
books_2015 = scrape_amazon_bestsellers(2015)

# Save to CSV
df = pd.DataFrame(books_2015)
df.to_csv("amazon_bestsellers_2015.csv", index=False)
driver.quit()
print("Saved data to amazon_bestsellers_2015.csv")

Scraping data for 2015...
Saved data to amazon_bestsellers_2015.csv


In [85]:
#2016FINAL COUNTDOWN

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

# Configure headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    # Try breadcrumbs method first
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    # Extract the customer average rating
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    if rating_tag:
        return rating_tag.get_text(strip=True)  # Extract the rating (e.g., 4.8)
    return 'N/A'

def extract_price(soup):
    # Extract price from the page
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        return price_span.get_text(strip=True) + "." + soup.select_one("span.a-price-fraction").get_text(strip=True)
    return 'N/A'  # Return 'N/A' if no price is found

def extract_reviews(soup):
    # Extract number of customer reviews
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    if reviews_tag:
        return reviews_tag.get_text(strip=True)
    return 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        # Use Open Library API to fetch book data based on the title
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()

        # Check if data is available
        if data['docs']:
            book_info = data['docs'][0]  # Take the first result (most relevant)
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception as e:
        print(f"Error fetching data from Open Library API for title: {title}")
        return 'Unknown', 'Unknown'

def extract_year_from_subtitle(soup):
    # Look for the product subtitle which contains the publishing date
    subtitle_tag = soup.select_one("span#productSubtitle")
    if subtitle_tag:
        subtitle_text = subtitle_tag.get_text(strip=True)
        # Extract the year from the subtitle text
        match = re.search(r"(\d{4})", subtitle_text)
        if match:
            return match.group(1)
    return 'Unknown'  # If no year is found, return 'Unknown'

def extract_year_and_price_from_amazon(soup):
    # Look for the div containing both the price and the potential year in the page structure
    price_div = soup.select_one("div#corePriceDisplay_desktop_feature_div")
    if price_div:
        # Extract the price
        price_tag = price_div.select_one("span.a-price span.a-offscreen")
        price = price_tag.get_text(strip=True) if price_tag else 'N/A'

        # Extract year from the text if available
        year_tag = price_div.select_one("span.a-size-base.a-color-secondary")
        if year_tag:
            year = year_tag.get_text(strip=True)
            # Check if the year and price are mixed (price + year)
            match = re.search(r"(\d{4})", year)
            if match:
                year = match.group(1)
            return year, price
    return 'Unknown', 'N/A'

def scrape_amazon_bestsellers(year):
    print(f"Scraping data for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 3):  # Scrape 2 pages as an example
        url = f"{base_url}?page={page}"
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y").get_text(strip=True)
                author = card.select_one("div.a-row.a-size-small").get_text(strip=True)
                book_url_tag = card.select_one("a.a-link-normal")
                book_url = "https://www.amazon.com" + book_url_tag['href']

                # Visit individual book page to get genre, rating, and price
                driver.get(book_url)
                time.sleep(1)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')
                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)  # Extract rating using updated function
                reviews = extract_reviews(book_soup)  # Extract the number of reviews

                # Fetch publisher and published year using Open Library API
                publisher, published_year = fetch_book_info_from_openlibrary(title)

                # If publisher and year are still unknown, try to extract from Amazon
                if published_year == 'Unknown':
                    # Extract year from the product subtitle
                    published_year = extract_year_from_subtitle(book_soup)
                    price = extract_price(book_soup)  # Fetch the price separately if not found together
                else:
                    price = extract_price(book_soup)  # Fetch the price separately if not found together

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping a book: {e}")
                continue

    return book_data

# Collect for a single year (e.g., 2016)
books_2016 = scrape_amazon_bestsellers(2016)

# Save to CSV
df = pd.DataFrame(books_2016)
df.to_csv("amazon_bestsellers_2016.csv", index=False)
driver.quit()
print("Saved data to amazon_bestsellers_2016.csv")

Scraping data for 2016...
Saved data to amazon_bestsellers_2016.csv


In [84]:
# 2017FINAL COUNTDOWN

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

# Configure headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    # Try breadcrumbs method first
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    # Extract the customer average rating
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    if rating_tag:
        return rating_tag.get_text(strip=True)  # Extract the rating (e.g., 4.8)
    return 'N/A'

def extract_price(soup):
    # Extract price from the page
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        return price_span.get_text(strip=True) + "." + soup.select_one("span.a-price-fraction").get_text(strip=True)
    return 'N/A'  # Return 'N/A' if no price is found

def extract_reviews(soup):
    # Extract number of customer reviews
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    if reviews_tag:
        return reviews_tag.get_text(strip=True)
    return 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        # Use Open Library API to fetch book data based on the title
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()

        # Check if data is available
        if data['docs']:
            book_info = data['docs'][0]  # Take the first result (most relevant)
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception as e:
        print(f"Error fetching data from Open Library API for title: {title}")
        return 'Unknown', 'Unknown'

def extract_year_from_subtitle(soup):
    # Look for the product subtitle which contains the publishing date
    subtitle_tag = soup.select_one("span#productSubtitle")
    if subtitle_tag:
        subtitle_text = subtitle_tag.get_text(strip=True)
        # Extract the year from the subtitle text
        match = re.search(r"(\d{4})", subtitle_text)
        if match:
            return match.group(1)
    return 'Unknown'  # If no year is found, return 'Unknown'

def extract_year_and_price_from_amazon(soup):
    # Look for the div containing both the price and the potential year in the page structure
    price_div = soup.select_one("div#corePriceDisplay_desktop_feature_div")
    if price_div:
        # Extract the price
        price_tag = price_div.select_one("span.a-price span.a-offscreen")
        price = price_tag.get_text(strip=True) if price_tag else 'N/A'

        # Extract year from the text if available
        year_tag = price_div.select_one("span.a-size-base.a-color-secondary")
        if year_tag:
            year = year_tag.get_text(strip=True)
            # Check if the year and price are mixed (price + year)
            match = re.search(r"(\d{4})", year)
            if match:
                year = match.group(1)
            return year, price
    return 'Unknown', 'N/A'

def scrape_amazon_bestsellers(year):
    print(f"Scraping data for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 3):  # Scrape 2 pages as an example
        url = f"{base_url}?page={page}"
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y").get_text(strip=True)
                author = card.select_one("div.a-row.a-size-small").get_text(strip=True)
                book_url_tag = card.select_one("a.a-link-normal")
                book_url = "https://www.amazon.com" + book_url_tag['href']

                # Visit individual book page to get genre, rating, and price
                driver.get(book_url)
                time.sleep(1)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')
                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)  # Extract rating using updated function
                reviews = extract_reviews(book_soup)  # Extract the number of reviews

                # Fetch publisher and published year using Open Library API
                publisher, published_year = fetch_book_info_from_openlibrary(title)

                # If publisher and year are still unknown, try to extract from Amazon
                if published_year == 'Unknown':
                    # Extract year from the product subtitle
                    published_year = extract_year_from_subtitle(book_soup)
                    price = extract_price(book_soup)  # Fetch the price separately if not found together
                else:
                    price = extract_price(book_soup)  # Fetch the price separately if not found together

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping a book: {e}")
                continue

    return book_data

# Collect for a single year (e.g., 2017)
books_2017 = scrape_amazon_bestsellers(2017)

# Save to CSV
df = pd.DataFrame(books_2017)
df.to_csv("amazon_bestsellers_2017.csv", index=False)
driver.quit()
print("Saved data to amazon_bestsellers_2017.csv")

Scraping data for 2017...
Saved data to amazon_bestsellers_2017.csv


In [83]:
# 2018 FINAL COUNTDOWN

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

# Configure headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    # Try breadcrumbs method first
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    # Extract the customer average rating
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    if rating_tag:
        return rating_tag.get_text(strip=True)  # Extract the rating (e.g., 4.8)
    return 'N/A'

def extract_price(soup):
    # Extract price from the page
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        return price_span.get_text(strip=True) + "." + soup.select_one("span.a-price-fraction").get_text(strip=True)
    return 'N/A'  # Return 'N/A' if no price is found

def extract_reviews(soup):
    # Extract number of customer reviews
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    if reviews_tag:
        return reviews_tag.get_text(strip=True)
    return 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        # Use Open Library API to fetch book data based on the title
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()

        # Check if data is available
        if data['docs']:
            book_info = data['docs'][0]  # Take the first result (most relevant)
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception as e:
        print(f"Error fetching data from Open Library API for title: {title}")
        return 'Unknown', 'Unknown'

def extract_year_from_subtitle(soup):
    # Look for the product subtitle which contains the publishing date
    subtitle_tag = soup.select_one("span#productSubtitle")
    if subtitle_tag:
        subtitle_text = subtitle_tag.get_text(strip=True)
        # Extract the year from the subtitle text
        match = re.search(r"(\d{4})", subtitle_text)
        if match:
            return match.group(1)
    return 'Unknown'  # If no year is found, return 'Unknown'

def extract_year_and_price_from_amazon(soup):
    # Look for the div containing both the price and the potential year in the page structure
    price_div = soup.select_one("div#corePriceDisplay_desktop_feature_div")
    if price_div:
        # Extract the price
        price_tag = price_div.select_one("span.a-price span.a-offscreen")
        price = price_tag.get_text(strip=True) if price_tag else 'N/A'

        # Extract year from the text if available
        year_tag = price_div.select_one("span.a-size-base.a-color-secondary")
        if year_tag:
            year = year_tag.get_text(strip=True)
            # Check if the year and price are mixed (price + year)
            match = re.search(r"(\d{4})", year)
            if match:
                year = match.group(1)
            return year, price
    return 'Unknown', 'N/A'

def scrape_amazon_bestsellers(year):
    print(f"Scraping data for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 3):  # Scrape 2 pages as an example
        url = f"{base_url}?page={page}"
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y").get_text(strip=True)
                author = card.select_one("div.a-row.a-size-small").get_text(strip=True)
                book_url_tag = card.select_one("a.a-link-normal")
                book_url = "https://www.amazon.com" + book_url_tag['href']

                # Visit individual book page to get genre, rating, and price
                driver.get(book_url)
                time.sleep(1)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')
                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)  # Extract rating using updated function
                reviews = extract_reviews(book_soup)  # Extract the number of reviews

                # Fetch publisher and published year using Open Library API
                publisher, published_year = fetch_book_info_from_openlibrary(title)

                # If publisher and year are still unknown, try to extract from Amazon
                if published_year == 'Unknown':
                    # Extract year from the product subtitle
                    published_year = extract_year_from_subtitle(book_soup)
                    price = extract_price(book_soup)  # Fetch the price separately if not found together
                else:
                    price = extract_price(book_soup)  # Fetch the price separately if not found together

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping a book: {e}")
                continue

    return book_data

# Collect for a single year (e.g., 2018)
books_2018 = scrape_amazon_bestsellers(2018)

# Save to CSV
df = pd.DataFrame(books_2018)
df.to_csv("amazon_bestsellers_2018.csv", index=False)
driver.quit()
print("Saved data to amazon_bestsellers_2018.csv")

Scraping data for 2018...
Saved data to amazon_bestsellers_2018.csv


In [82]:
#FINAL COUNTDOWN

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

# Configure headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    # Try breadcrumbs method first
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    # Extract the customer average rating
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    if rating_tag:
        return rating_tag.get_text(strip=True)  # Extract the rating (e.g., 4.8)
    return 'N/A'

def extract_price(soup):
    # Extract price from the page
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        return price_span.get_text(strip=True) + "." + soup.select_one("span.a-price-fraction").get_text(strip=True)
    return 'N/A'  # Return 'N/A' if no price is found

def extract_reviews(soup):
    # Extract number of customer reviews
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    if reviews_tag:
        return reviews_tag.get_text(strip=True)
    return 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        # Use Open Library API to fetch book data based on the title
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()

        # Check if data is available
        if data['docs']:
            book_info = data['docs'][0]  # Take the first result (most relevant)
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception as e:
        print(f"Error fetching data from Open Library API for title: {title}")
        return 'Unknown', 'Unknown'

def extract_year_from_subtitle(soup):
    # Look for the product subtitle which contains the publishing date
    subtitle_tag = soup.select_one("span#productSubtitle")
    if subtitle_tag:
        subtitle_text = subtitle_tag.get_text(strip=True)
        # Extract the year from the subtitle text
        match = re.search(r"(\d{4})", subtitle_text)
        if match:
            return match.group(1)
    return 'Unknown'  # If no year is found, return 'Unknown'

def extract_year_and_price_from_amazon(soup):
    # Look for the div containing both the price and the potential year in the page structure
    price_div = soup.select_one("div#corePriceDisplay_desktop_feature_div")
    if price_div:
        # Extract the price
        price_tag = price_div.select_one("span.a-price span.a-offscreen")
        price = price_tag.get_text(strip=True) if price_tag else 'N/A'

        # Extract year from the text if available
        year_tag = price_div.select_one("span.a-size-base.a-color-secondary")
        if year_tag:
            year = year_tag.get_text(strip=True)
            # Check if the year and price are mixed (price + year)
            match = re.search(r"(\d{4})", year)
            if match:
                year = match.group(1)
            return year, price
    return 'Unknown', 'N/A'

def scrape_amazon_bestsellers(year):
    print(f"Scraping data for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 3):  # Scrape 2 pages as an example
        url = f"{base_url}?page={page}"
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y").get_text(strip=True)
                author = card.select_one("div.a-row.a-size-small").get_text(strip=True)
                book_url_tag = card.select_one("a.a-link-normal")
                book_url = "https://www.amazon.com" + book_url_tag['href']

                # Visit individual book page to get genre, rating, and price
                driver.get(book_url)
                time.sleep(1)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')
                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)  # Extract rating using updated function
                reviews = extract_reviews(book_soup)  # Extract the number of reviews

                # Fetch publisher and published year using Open Library API
                publisher, published_year = fetch_book_info_from_openlibrary(title)

                # If publisher and year are still unknown, try to extract from Amazon
                if published_year == 'Unknown':
                    # Extract year from the product subtitle
                    published_year = extract_year_from_subtitle(book_soup)
                    price = extract_price(book_soup)  # Fetch the price separately if not found together
                else:
                    price = extract_price(book_soup)  # Fetch the price separately if not found together

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping a book: {e}")
                continue

    return book_data

# Collect for a single year (e.g., 2019)
books_2019 = scrape_amazon_bestsellers(2019)

# Save to CSV
df = pd.DataFrame(books_2019)
df.to_csv("amazon_bestsellers_2019.csv", index=False)
driver.quit()
print("Saved data to amazon_bestsellers_2019.csv")

Scraping data for 2019...
Error scraping a book: 'NoneType' object has no attribute 'get_text'
Error scraping a book: 'NoneType' object has no attribute 'get_text'
Saved data to amazon_bestsellers_2019.csv


In [ ]:
#FINAL COUNTDOWN

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

# Configure headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    # Try breadcrumbs method first
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    # Extract the customer average rating
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    if rating_tag:
        return rating_tag.get_text(strip=True)  # Extract the rating (e.g., 4.8)
    return 'N/A'

def extract_price(soup):
    # Extract price from the page
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        return price_span.get_text(strip=True) + "." + soup.select_one("span.a-price-fraction").get_text(strip=True)
    return 'N/A'  # Return 'N/A' if no price is found

def extract_reviews(soup):
    # Extract number of customer reviews
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    if reviews_tag:
        return reviews_tag.get_text(strip=True)
    return 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        # Use Open Library API to fetch book data based on the title
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()

        # Check if data is available
        if data['docs']:
            book_info = data['docs'][0]  # Take the first result (most relevant)
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception as e:
        print(f"Error fetching data from Open Library API for title: {title}")
        return 'Unknown', 'Unknown'

def extract_year_from_subtitle(soup):
    # Look for the product subtitle which contains the publishing date
    subtitle_tag = soup.select_one("span#productSubtitle")
    if subtitle_tag:
        subtitle_text = subtitle_tag.get_text(strip=True)
        # Extract the year from the subtitle text
        match = re.search(r"(\d{4})", subtitle_text)
        if match:
            return match.group(1)
    return 'Unknown'  # If no year is found, return 'Unknown'

def extract_year_and_price_from_amazon(soup):
    # Look for the div containing both the price and the potential year in the page structure
    price_div = soup.select_one("div#corePriceDisplay_desktop_feature_div")
    if price_div:
        # Extract the price
        price_tag = price_div.select_one("span.a-price span.a-offscreen")
        price = price_tag.get_text(strip=True) if price_tag else 'N/A'

        # Extract year from the text if available
        year_tag = price_div.select_one("span.a-size-base.a-color-secondary")
        if year_tag:
            year = year_tag.get_text(strip=True)
            # Check if the year and price are mixed (price + year)
            match = re.search(r"(\d{4})", year)
            if match:
                year = match.group(1)
            return year, price
    return 'Unknown', 'N/A'

def scrape_amazon_bestsellers(year):
    print(f"Scraping data for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 3):  # Scrape 2 pages as an example
        url = f"{base_url}?page={page}"
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y").get_text(strip=True)
                author = card.select_one("div.a-row.a-size-small").get_text(strip=True)
                book_url_tag = card.select_one("a.a-link-normal")
                book_url = "https://www.amazon.com" + book_url_tag['href']

                # Visit individual book page to get genre, rating, and price
                driver.get(book_url)
                time.sleep(1)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')
                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)  # Extract rating using updated function
                reviews = extract_reviews(book_soup)  # Extract the number of reviews

                # Fetch publisher and published year using Open Library API
                publisher, published_year = fetch_book_info_from_openlibrary(title)

                # If publisher and year are still unknown, try to extract from Amazon
                if published_year == 'Unknown':
                    # Extract year from the product subtitle
                    published_year = extract_year_from_subtitle(book_soup)
                    price = extract_price(book_soup)  # Fetch the price separately if not found together
                else:
                    price = extract_price(book_soup)  # Fetch the price separately if not found together

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping a book: {e}")
                continue

    return book_data

# Collect for a single year (e.g., 2020)
books_2020 = scrape_amazon_bestsellers(2020)

# Save to CSV
df = pd.DataFrame(books_2020)
df.to_csv("amazon_bestsellers_2020.csv", index=False)
driver.quit()
print("Saved data to amazon_bestsellers_2020.csv")


Scraping data for 2020...
Saved data to amazon_bestsellers_2020.csv


In [70]:
#2021

# Configure headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    # Try breadcrumbs method first
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    # Extract the customer average rating
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    if rating_tag:
        return rating_tag.get_text(strip=True)  # Extract the rating (e.g., 4.8)
    return 'N/A'

def extract_price(soup):
    # Extract price from the page
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        return price_span.get_text(strip=True) + "." + soup.select_one("span.a-price-fraction").get_text(strip=True)
    return 'N/A'  # Return 'N/A' if no price is found

def extract_reviews(soup):
    # Extract number of customer reviews
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    if reviews_tag:
        return reviews_tag.get_text(strip=True)
    return 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        # Use Open Library API to fetch book data based on the title
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()

        # Check if data is available
        if data['docs']:
            book_info = data['docs'][0]  # Take the first result (most relevant)
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception as e:
        print(f"Error fetching data from Open Library API for title: {title}")
        return 'Unknown', 'Unknown'

def extract_year_from_subtitle(soup):
    # Look for the product subtitle which contains the publishing date
    subtitle_tag = soup.select_one("span#productSubtitle")
    if subtitle_tag:
        subtitle_text = subtitle_tag.get_text(strip=True)
        # Extract the year from the subtitle text
        match = re.search(r"(\d{4})", subtitle_text)
        if match:
            return match.group(1)
    return 'Unknown'  # If no year is found, return 'Unknown'

def extract_year_and_price_from_amazon(soup):
    # Look for the div containing both the price and the potential year in the page structure
    price_div = soup.select_one("div#corePriceDisplay_desktop_feature_div")
    if price_div:
        # Extract the price
        price_tag = price_div.select_one("span.a-price span.a-offscreen")
        price = price_tag.get_text(strip=True) if price_tag else 'N/A'

        # Extract year from the text if available
        year_tag = price_div.select_one("span.a-size-base.a-color-secondary")
        if year_tag:
            year = year_tag.get_text(strip=True)
            # Check if the year and price are mixed (price + year)
            match = re.search(r"(\d{4})", year)
            if match:
                year = match.group(1)
            return year, price
    return 'Unknown', 'N/A'

def scrape_amazon_bestsellers(year):
    print(f"Scraping data for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 3):  # Scrape 2 pages as an example
        url = f"{base_url}?page={page}"
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y").get_text(strip=True)
                author = card.select_one("div.a-row.a-size-small").get_text(strip=True)
                book_url_tag = card.select_one("a.a-link-normal")
                book_url = "https://www.amazon.com" + book_url_tag['href']

                # Visit individual book page to get genre, rating, and price
                driver.get(book_url)
                time.sleep(1)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')
                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)  # Extract rating using updated function
                reviews = extract_reviews(book_soup)  # Extract the number of reviews

                # Fetch publisher and published year using Open Library API
                publisher, published_year = fetch_book_info_from_openlibrary(title)

                # If publisher and year are still unknown, try to extract from Amazon
                if published_year == 'Unknown':
                    # Extract year from the product subtitle
                    published_year = extract_year_from_subtitle(book_soup)
                    price = extract_price(book_soup)  # Fetch the price separately if not found together
                else:
                    price = extract_price(book_soup)  # Fetch the price separately if not found together

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping a book: {e}")
                continue

    return book_data

# Collect for a single year (e.g., 2021)
books_2021 = scrape_amazon_bestsellers(2021)

# Save to CSV
df = pd.DataFrame(books_2021)
df.to_csv("amazon_bestsellers_2021.csv", index=False)
driver.quit()
print("Saved data to amazon_bestsellers_2021.csv")

Scraping data for 2021...
Saved data to amazon_bestsellers_2021.csv


In [71]:
#2022
#  Configure headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    # Try breadcrumbs method first
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    # Extract the customer average rating
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    if rating_tag:
        return rating_tag.get_text(strip=True)  # Extract the rating (e.g., 4.8)
    return 'N/A'

def extract_price(soup):
    # Extract price from the page
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        return price_span.get_text(strip=True) + "." + soup.select_one("span.a-price-fraction").get_text(strip=True)
    return 'N/A'  # Return 'N/A' if no price is found

def extract_reviews(soup):
    # Extract number of customer reviews
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    if reviews_tag:
        return reviews_tag.get_text(strip=True)
    return 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        # Use Open Library API to fetch book data based on the title
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()

        # Check if data is available
        if data['docs']:
            book_info = data['docs'][0]  # Take the first result (most relevant)
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception as e:
        print(f"Error fetching data from Open Library API for title: {title}")
        return 'Unknown', 'Unknown'

def extract_year_from_subtitle(soup):
    # Look for the product subtitle which contains the publishing date
    subtitle_tag = soup.select_one("span#productSubtitle")
    if subtitle_tag:
        subtitle_text = subtitle_tag.get_text(strip=True)
        # Extract the year from the subtitle text
        match = re.search(r"(\d{4})", subtitle_text)
        if match:
            return match.group(1)
    return 'Unknown'  # If no year is found, return 'Unknown'

def extract_year_and_price_from_amazon(soup):
    # Look for the div containing both the price and the potential year in the page structure
    price_div = soup.select_one("div#corePriceDisplay_desktop_feature_div")
    if price_div:
        # Extract the price
        price_tag = price_div.select_one("span.a-price span.a-offscreen")
        price = price_tag.get_text(strip=True) if price_tag else 'N/A'

        # Extract year from the text if available
        year_tag = price_div.select_one("span.a-size-base.a-color-secondary")
        if year_tag:
            year = year_tag.get_text(strip=True)
            # Check if the year and price are mixed (price + year)
            match = re.search(r"(\d{4})", year)
            if match:
                year = match.group(1)
            return year, price
    return 'Unknown', 'N/A'

def scrape_amazon_bestsellers(year):
    print(f"Scraping data for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 3):  # Scrape 2 pages as an example
        url = f"{base_url}?page={page}"
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y").get_text(strip=True)
                author = card.select_one("div.a-row.a-size-small").get_text(strip=True)
                book_url_tag = card.select_one("a.a-link-normal")
                book_url = "https://www.amazon.com" + book_url_tag['href']

                # Visit individual book page to get genre, rating, and price
                driver.get(book_url)
                time.sleep(1)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')
                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)  # Extract rating using updated function
                reviews = extract_reviews(book_soup)  # Extract the number of reviews

                # Fetch publisher and published year using Open Library API
                publisher, published_year = fetch_book_info_from_openlibrary(title)

                # If publisher and year are still unknown, try to extract from Amazon
                if published_year == 'Unknown':
                    # Extract year from the product subtitle
                    published_year = extract_year_from_subtitle(book_soup)
                    price = extract_price(book_soup)  # Fetch the price separately if not found together
                else:
                    price = extract_price(book_soup)  # Fetch the price separately if not found together

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping a book: {e}")
                continue

    return book_data

# Collect for a single year (e.g., 2022)
books_2022 = scrape_amazon_bestsellers(2022)

# Save to CSV
df = pd.DataFrame(books_2022)
df.to_csv("amazon_bestsellers_2022.csv", index=False)
driver.quit()
print("Saved data to amazon_bestsellers_2022.csv")

Scraping data for 2022...
Saved data to amazon_bestsellers_2022.csv


In [72]:
# 2023
# Configure headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    # Try breadcrumbs method first
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    # Extract the customer average rating
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    if rating_tag:
        return rating_tag.get_text(strip=True)  # Extract the rating (e.g., 4.8)
    return 'N/A'

def extract_price(soup):
    # Extract price from the page
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        return price_span.get_text(strip=True) + "." + soup.select_one("span.a-price-fraction").get_text(strip=True)
    return 'N/A'  # Return 'N/A' if no price is found

def extract_reviews(soup):
    # Extract number of customer reviews
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    if reviews_tag:
        return reviews_tag.get_text(strip=True)
    return 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        # Use Open Library API to fetch book data based on the title
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()

        # Check if data is available
        if data['docs']:
            book_info = data['docs'][0]  # Take the first result (most relevant)
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception as e:
        print(f"Error fetching data from Open Library API for title: {title}")
        return 'Unknown', 'Unknown'

def extract_year_from_subtitle(soup):
    # Look for the product subtitle which contains the publishing date
    subtitle_tag = soup.select_one("span#productSubtitle")
    if subtitle_tag:
        subtitle_text = subtitle_tag.get_text(strip=True)
        # Extract the year from the subtitle text
        match = re.search(r"(\d{4})", subtitle_text)
        if match:
            return match.group(1)
    return 'Unknown'  # If no year is found, return 'Unknown'

def extract_year_and_price_from_amazon(soup):
    # Look for the div containing both the price and the potential year in the page structure
    price_div = soup.select_one("div#corePriceDisplay_desktop_feature_div")
    if price_div:
        # Extract the price
        price_tag = price_div.select_one("span.a-price span.a-offscreen")
        price = price_tag.get_text(strip=True) if price_tag else 'N/A'

        # Extract year from the text if available
        year_tag = price_div.select_one("span.a-size-base.a-color-secondary")
        if year_tag:
            year = year_tag.get_text(strip=True)
            # Check if the year and price are mixed (price + year)
            match = re.search(r"(\d{4})", year)
            if match:
                year = match.group(1)
            return year, price
    return 'Unknown', 'N/A'

def scrape_amazon_bestsellers(year):
    print(f"Scraping data for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 3):  # Scrape 2 pages as an example
        url = f"{base_url}?page={page}"
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y").get_text(strip=True)
                author = card.select_one("div.a-row.a-size-small").get_text(strip=True)
                book_url_tag = card.select_one("a.a-link-normal")
                book_url = "https://www.amazon.com" + book_url_tag['href']

                # Visit individual book page to get genre, rating, and price
                driver.get(book_url)
                time.sleep(1)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')
                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)  # Extract rating using updated function
                reviews = extract_reviews(book_soup)  # Extract the number of reviews

                # Fetch publisher and published year using Open Library API
                publisher, published_year = fetch_book_info_from_openlibrary(title)

                # If publisher and year are still unknown, try to extract from Amazon
                if published_year == 'Unknown':
                    # Extract year from the product subtitle
                    published_year = extract_year_from_subtitle(book_soup)
                    price = extract_price(book_soup)  # Fetch the price separately if not found together
                else:
                    price = extract_price(book_soup)  # Fetch the price separately if not found together

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping a book: {e}")
                continue

    return book_data

# Collect for a single year (e.g., 2023)
books_2023 = scrape_amazon_bestsellers(2023)

# Save to CSV
df = pd.DataFrame(books_2023)
df.to_csv("amazon_bestsellers_2023.csv", index=False)
driver.quit()
print("Saved data to amazon_bestsellers_2023.csv")

Scraping data for 2023...
Saved data to amazon_bestsellers_2023.csv


In [73]:
# 2024
# Configure headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    # Try breadcrumbs method first
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    # Extract the customer average rating
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    if rating_tag:
        return rating_tag.get_text(strip=True)  # Extract the rating (e.g., 4.8)
    return 'N/A'

def extract_price(soup):
    # Extract price from the page
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        return price_span.get_text(strip=True) + "." + soup.select_one("span.a-price-fraction").get_text(strip=True)
    return 'N/A'  # Return 'N/A' if no price is found

def extract_reviews(soup):
    # Extract number of customer reviews
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    if reviews_tag:
        return reviews_tag.get_text(strip=True)
    return 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        # Use Open Library API to fetch book data based on the title
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()

        # Check if data is available
        if data['docs']:
            book_info = data['docs'][0]  # Take the first result (most relevant)
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception as e:
        print(f"Error fetching data from Open Library API for title: {title}")
        return 'Unknown', 'Unknown'

def extract_year_from_subtitle(soup):
    # Look for the product subtitle which contains the publishing date
    subtitle_tag = soup.select_one("span#productSubtitle")
    if subtitle_tag:
        subtitle_text = subtitle_tag.get_text(strip=True)
        # Extract the year from the subtitle text
        match = re.search(r"(\d{4})", subtitle_text)
        if match:
            return match.group(1)
    return 'Unknown'  # If no year is found, return 'Unknown'

def extract_year_and_price_from_amazon(soup):
    # Look for the div containing both the price and the potential year in the page structure
    price_div = soup.select_one("div#corePriceDisplay_desktop_feature_div")
    if price_div:
        # Extract the price
        price_tag = price_div.select_one("span.a-price span.a-offscreen")
        price = price_tag.get_text(strip=True) if price_tag else 'N/A'

        # Extract year from the text if available
        year_tag = price_div.select_one("span.a-size-base.a-color-secondary")
        if year_tag:
            year = year_tag.get_text(strip=True)
            # Check if the year and price are mixed (price + year)
            match = re.search(r"(\d{4})", year)
            if match:
                year = match.group(1)
            return year, price
    return 'Unknown', 'N/A'

def scrape_amazon_bestsellers(year):
    print(f"Scraping data for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 3):  # Scrape 2 pages as an example
        url = f"{base_url}?page={page}"
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards: 
            try:
                title = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y").get_text(strip=True) #From HTML tag in amazon
                author = card.select_one("div.a-row.a-size-small").get_text(strip=True) #From HTML tag in amazon
                book_url_tag = card.select_one("a.a-link-normal")
                book_url = "https://www.amazon.com" + book_url_tag['href']

                # Visit individual book page to get genre, rating, and price
                driver.get(book_url)
                time.sleep(1)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')
                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)  # Extract rating using updated function
                reviews = extract_reviews(book_soup)  # Extract the number of reviews

                # Fetch publisher and published year using Open Library API
                publisher, published_year = fetch_book_info_from_openlibrary(title)

                # If publisher and year are still unknown, try to extract from Amazon
                if published_year == 'Unknown':
                    # Extract year from the product subtitle
                    published_year = extract_year_from_subtitle(book_soup)
                    price = extract_price(book_soup)  # Fetch the price separately if not found together
                else:
                    price = extract_price(book_soup)  # Fetch the price separately if not found together

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping a book: {e}")
                continue

    return book_data

# Collect for a single year (e.g., 2024)
books_2024 = scrape_amazon_bestsellers(2024)

# Save to CSV
df = pd.DataFrame(books_2024)
df.to_csv("amazon_bestsellers_2024.csv", index=False)
driver.quit()
print("Saved data to amazon_bestsellers_2024.csv")

Scraping data for 2024...
Saved data to amazon_bestsellers_2024.csv


In [74]:
# 2025
# Configure headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    # Try breadcrumbs method first
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    # Extract the customer average rating
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    if rating_tag:
        return rating_tag.get_text(strip=True)  # Extract the rating (e.g., 4.8)
    return 'N/A'

def extract_price(soup):
    # Extract price from the page
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        return price_span.get_text(strip=True) + "." + soup.select_one("span.a-price-fraction").get_text(strip=True)
    return 'N/A'  # Return 'N/A' if no price is found

def extract_reviews(soup):
    # Extract number of customer reviews
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    if reviews_tag:
        return reviews_tag.get_text(strip=True)
    return 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        # Use Open Library API to fetch book data based on the title
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()

        # Check if data is available
        if data['docs']:
            book_info = data['docs'][0]  # Take the first result (most relevant)
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception as e:
        print(f"Error fetching data from Open Library API for title: {title}")
        return 'Unknown', 'Unknown'

def extract_year_from_subtitle(soup):
    # Look for the product subtitle which contains the publishing date
    subtitle_tag = soup.select_one("span#productSubtitle")
    if subtitle_tag:
        subtitle_text = subtitle_tag.get_text(strip=True)
        # Extract the year from the subtitle text
        match = re.search(r"(\d{4})", subtitle_text)
        if match:
            return match.group(1)
    return 'Unknown'  # If no year is found, return 'Unknown'

def extract_year_and_price_from_amazon(soup):
    # Look for the div containing both the price and the potential year in the page structure
    price_div = soup.select_one("div#corePriceDisplay_desktop_feature_div")
    if price_div:
        # Extract the price
        price_tag = price_div.select_one("span.a-price span.a-offscreen")
        price = price_tag.get_text(strip=True) if price_tag else 'N/A'

        # Extract year from the text if available
        year_tag = price_div.select_one("span.a-size-base.a-color-secondary")
        if year_tag:
            year = year_tag.get_text(strip=True)
            # Check if the year and price are mixed (price + year)
            match = re.search(r"(\d{4})", year)
            if match:
                year = match.group(1)
            return year, price
    return 'Unknown', 'N/A'

def scrape_amazon_bestsellers(year):
    print(f"Scraping data for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 3):  # Scrape 2 pages as an example
        url = f"{base_url}?page={page}"
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y").get_text(strip=True)
                author = card.select_one("div.a-row.a-size-small").get_text(strip=True)
                book_url_tag = card.select_one("a.a-link-normal")
                book_url = "https://www.amazon.com" + book_url_tag['href']

                # Visit individual book page to get genre, rating, and price
                driver.get(book_url)
                time.sleep(1)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')
                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)  # Extract rating using updated function
                reviews = extract_reviews(book_soup)  # Extract the number of reviews

                # Fetch publisher and published year using Open Library API
                publisher, published_year = fetch_book_info_from_openlibrary(title)

                # If publisher and year are still unknown, try to extract from Amazon
                if published_year == 'Unknown':
                    # Extract year from the product subtitle
                    published_year = extract_year_from_subtitle(book_soup)
                    price = extract_price(book_soup)  # Fetch the price separately if not found together
                else:
                    price = extract_price(book_soup)  # Fetch the price separately if not found together

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping a book: {e}")
                continue

    return book_data

# Collect for a single year (e.g., 2025)
books_2025 = scrape_amazon_bestsellers(2025)

# Save to CSV
df = pd.DataFrame(books_2025)
df.to_csv("amazon_bestsellers_2025.csv", index=False)
driver.quit()
print("Saved data to amazon_bestsellers_2025.csv")

Scraping data for 2025...
Saved data to amazon_bestsellers_2025.csv


In [97]:
import pandas as pd
import glob

# Load all CSV files from 2015 to 2025
years = range(2015, 2026)
file_names = [f"amazon_bestsellers_{year}.csv" for year in years]

# Read and combine all dataframes
df_list = [pd.read_csv(file) for file in file_names]
combined_df = pd.concat(df_list, ignore_index=True)

# Save to a new CSV
combined_df.to_csv("amazon_bestsellers_2015_2025.csv", index=False)

print("Combined CSV saved as amazon_bestsellers_2015_2025.csv")

Combined CSV saved as amazon_bestsellers_2015_2025.csv


In [100]:
df = pd.read_csv("amazon_bestsellers_2015_2025.csv")

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        360 non-null    object 
 1   author       360 non-null    object 
 2   genre        360 non-null    object 
 3   rating       360 non-null    float64
 4   price        360 non-null    object 
 5   year         360 non-null    int64  
 6   reviews      360 non-null    object 
 7   amazon_year  360 non-null    int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 22.6+ KB


In [101]:
df.head(
    
)

,title,author,genre,rating,price,year,reviews,amazon_year
0,First 100 Words: A Padded Board Book,Roger Priddy,Vocabulary & Spelling,4.7,3..88,2011,"38,702 ratings",2015
1,The Life-Changing Magic of Tidying Up: The Jap...,Marie Kondō,"Cleaning, Caretaking & Relocating",4.5,9..42,2017,"59,251 ratings",2015
2,Laugh-Out-Loud Jokes for Kids,Rob Elliott,Humorous,4.6,4..64,2022,"12,124 ratings",2015
3,Secret Garden: An Inky Treasure Hunt and Color...,Johanna Basford,Use of Color,4.7,10..18,2013,"15,936 ratings",2015
4,StrengthsFinder 2.0,Gallup,Management,4.1,28..52,2007,"8,423 ratings",2015


In [102]:
df.head(50)

,title,author,genre,rating,price,year,reviews,amazon_year
0,First 100 Words: A Padded Board Book,Roger Priddy,Vocabulary & Spelling,4.7,3..88,2011,"38,702 ratings",2015
1,The Life-Changing Magic of Tidying Up: The Jap...,Marie Kondō,"Cleaning, Caretaking & Relocating",4.5,9..42,2017,"59,251 ratings",2015
2,Laugh-Out-Loud Jokes for Kids,Rob Elliott,Humorous,4.6,4..64,2022,"12,124 ratings",2015
3,Secret Garden: An Inky Treasure Hunt and Color...,Johanna Basford,Use of Color,4.7,10..18,2013,"15,936 ratings",2015
4,StrengthsFinder 2.0,Gallup,Management,4.1,28..52,2007,"8,423 ratings",2015
5,Go Set a Watchman: A Novel,Harper Lee,Cultural Heritage,3.8,12..95,2018,"54,070 ratings",2015
6,Enchanted Forest: An Inky Quest and Coloring b...,Johanna Basford,Coloring Books,4.8,14..19,2015,"8,419 ratings",2015
7,All the Light We Cannot See,Anthony Doerr,World War II & Holocaust,4.5,12..93,2014,"232,429 ratings",2015
8,The Girl on the Train,Paula Hawkins,Suspense,4.1,12..19,2014,"494,408 ratings",2015
9,Adult Coloring Book: Stress Relieving Patterns,Blue Star Coloring,Use of Color,4.5,9..95,2015,"4,363 ratings",2015
